In [1]:
def RecursiveDFT(a, omega):
    n = len(a)
    if n == 1:
        return a
    a_even = [a[2*i] for i in range(n//2)] #or vector([a[i] for i in range(0, n, 2)])
    a_odd = [a[2*i + 1] for i in range(n//2)] #or a[1::2]
    
    hat_even = RecursiveDFT(a_even, omega^2)
    hat_odd = RecursiveDFT(a_odd, omega^2)

    w = 1
    hat_a_first_half = []
    hat_a_second_half = []
    for i in range(n//2):
        hat_a_first_half += [hat_even[i] + w*hat_odd[i]]
        hat_a_second_half += [hat_even[i] - w*hat_odd[i]]
        w *= omega
    return hat_a_first_half + hat_a_second_half

In [2]:
def InverseRecursiveDFT(hat_a, omega):
    n = len(hat_a)
    v = RecursiveDFT(hat_a, omega)
    a = [v_i / n for v_i in v]
    return a

In [3]:
import random

def sanity_check():
    # Setup
    k = 4
    n = 2**k

    # Generate a random list of unique integers within a range
    a = random.sample(range(1, 20), n)
    
    n = len(a)
    omega_n = E(n) # obtaining the n-th root of unity from https://doc.sagemath.org/html/en/reference/number_fields/sage/rings/universal_cyclotomic_field.html

    print("Everything running correctly:\n>>", InverseRecursiveDFT(RecursiveDFT(a,omega_n),omega_n ** (-1)) == a)

In [4]:
sanity_check()

Everything running correctly:
>> True


(c) Write a function **DFT_product** that computes the product of two integers, using fast Fourier transform. 

Hint: we do not require you to necessarily implement Schönhage-Strassen’s algorithm, but you can use the idea that $2$ is a primitive $2n$-th root of unity in $\mathbb{Z}/(2^n+1)\mathbb{Z}$. Otherwise working with complex roots of unity is fine.

In [5]:
def next_power_2(n):
    power = 1
    while power < n:
        power *= 2
    return power


def DFT_product(r, s, base=10, modulo=1):
    a = ZZ(r).digits(base)
    b = ZZ(s).digits(base)

    n = next_power_2(max(len(a), len(b), 2*base.nbits()))
    a += [0]*(2*n - len(a))#or a = r.digits(base=2, padto=2∗n)
    b += [0]*(2*n - len(b))

    if modulo:
        N = 2 ** n + 1
        omega_2n = mod(2, N)
    else:
        omega_2n = E(2*n)
    
    hat_a = RecursiveDFT(a, omega_2n)
    hat_b = RecursiveDFT(b, omega_2n)

    hat_prod = [hat_a[j]*hat_b[j] for j in range(2*n)] 
    prod = InverseRecursiveDFT(hat_prod, omega_2n **(-1))
    
    if modulo:
        return sum([prod[i].lift() * (base ** i) for i in range(2*n)])
    else:
        return sum([prod[i] * (base**i) for i in range(2*n)])

In [9]:
for r in range(50):
    for s in range(r, 50):
        #prod = DFT_product(r, s)
        #prod = DFT_product(r, s, base = 2)
        prod = DFT_product(r, s, base = 10, modulo = 0) #longer but it works
        if(r*s != prod):
            print(r, s, prod)